In [74]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs236299-2020/lab3-3.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [75]:
# Initialize Otter
import otter
grader = otter.Notebook()

%%latex
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\cnt}[1]{\sharp(#1)}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\softmax}{\operatorname{softmax}}
\newcommand{\Prob}{\Pr}
\newcommand{\given}{\,|\,}

$$
\renewcommand{\vect}[1]{\mathbf{#1}}
\renewcommand{\cnt}[1]{\sharp(#1)}
\renewcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\renewcommand{\softmax}{\operatorname{softmax}}
\renewcommand{\Prob}{\Pr}
\renewcommand{\given}{\,|\,}
$$

# Course 236299
## Lab 3-3 - Probabilistic context-free grammars

In previous labs, you have practiced constituency parsing using context-free grammars with the CKY parsing algorithm. In this lab you will extend this framework to a probabilistic one, probabilistic context-free grammars (PCFG).

New bits of Python used for the first time in the _solution set_ for this lab, and which you may therefore find useful:

* [`math.prod`](https://docs.python.org/3/library/math.html#math.prod)
* [`nltk.tree.Tree.productions`](https://www.nltk.org/api/nltk.html?highlight=production#nltk.tree.Tree.productions)

## Preparations

In [76]:
import copy
import math
import nltk
import pandas as pd

from collections import Counter, defaultdict
from pprint import pprint

## Syntactic ambiguity

Let's start with the following grammar for arithmetic word expressions:

In [77]:
arithmetic_grammar = nltk.CFG.fromstring("""
    S -> NUM | NUM OP S | S OP NUM
    NUM -> 'one' | 'two' | 'three' | 'four' | 'five' 
    NUM -> 'six' | 'seven' | 'eight' | 'nine' | 'ten' 
    OP -> ADD | SUB | MULT | DIV
    ADD -> 'plus' | 'added' 'to'
    SUB -> 'minus' 
    MULT -> 'times' | 'multiplied' 'by'
    DIV -> 'divided' 'by'
""")

>   It might have been more natural to have the rules
>   ```
>   S -> NUM | S OP S
>   ```
>   but we have purposefully introduced an ambiguity into the grammar via these more restricted rules above so that we can disambiguate later by rule weightings.

We can use the given CFG to parse the phrase "two plus three times four" and print the possible parse trees.

In [78]:
parser = nltk.parse.BottomUpChartParser(arithmetic_grammar)
phrase = "two plus three times four"
parses = list(parser.parse(phrase.split()))

for i, tree in enumerate(parses):
  print(f"Possible parse {i+1}:\n")
  tree.pretty_print()

Possible parse 1:

           S             
      _____|__________    
     S           |    |  
  ___|_____      |    |   
 |   OP    S     OP   |  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    |   
two plus three times four

Possible parse 2:

           S             
      _____|__________    
     S           |    |  
  ___|_____      |    |   
 S   OP    |     OP   |  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    |   
two plus three times four

Possible parse 3:

           S             
  _________|_____         
 |   |           S       
 |   |      _____|____    
 |   OP    |     OP   S  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    |   
two plus three times four

Possible parse 4:

           S             
  _________|_____         
 |   |           S       
 |   |      _____|____    
 |   OP    S     OP   |  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    

In this example, every parse tree represents an arithmetic expression. Manually calculate the value of the resulting equation for each of the parsed trees.

<!--
BEGIN QUESTION
name: parsed_equation_result
-->

In [79]:
#TODO
result_tree1 = 20
result_tree2 = 20
result_tree3 = 12
result_tree4 = 12

In [80]:
grader.check("parsed_equation_result")

All tests passed!

As you can see, we got four different parse trees, yielding different numerical results. It is interesting to notice that some of the parse trees (the pairs (1, 2) and (3, 4)) have different structures but the same "meaning" (same denoted value), while for other trees different structures induce different meanings (for example the pair (1, 3)).

The idea of different structural interpretations is called structural ambiguity. Since natural language is oftentimes ambiguous, this is a very realistic concern. One approach to deal with this issue is by defining a scoring system to score the possible parses and choosing the highest scoring tree. We will see how this can be done by taking a probabilistic approach to CFG.

## Probabilistic context-free grammars

To assign probabilities to strings, we will use a probabilistic context-free grammar (PCFG), a CFG in which each rule is augmented with a probability.

The PCFG definition is derived from that of the CFG, by augmenting the rules with probabilities:

* $\cal{N}$ – a set of nonterminal symbols
* $\Sigma$ – a set of terminal symbols
* $\cal{R}$ – a set of rules or productions, each of the form $A \rightarrow \beta\ [p]$,
where $A$ is a nonterminal, $\beta$ is a string of terminal or nonterminal symbols,
and $p$ is a number between 0 and 1 expressing $\Prob(\beta \given A)$
* $S$ – a designated start symbol

Note that to constitute a valid probability distribution we require that $\sum_\beta \Prob(\beta \given A) =1$, that is, the probabilities associated with all rules with the same left-hand side must sum to one.

Take a look at the following PCFG based on the arithmetic grammar above:

In [81]:
probabilistic_arithmetic_grammar = nltk.PCFG.fromstring("""
    S -> NUM [0.35] | NUM OP S [0.4] | S OP NUM [0.25] 
    OP -> ADD [0.4] | SUB [0.2] | MULT [0.3] | DIV [0.1]
    NUM -> 'one' [0.1] | 'two' [0.1] | 'three' [0.1] | 'four' [0.1] | 'five' [0.1]
    NUM -> 'six' [0.1] | 'seven' [0.1] | 'eight' [0.1] | 'nine' [0.1] | 'ten' [0.1]
    ADD -> 'plus' [0.8] | 'added' 'to' [0.2]
    SUB -> 'minus' [1.0]
    MULT -> 'times' [0.9] | 'multiplied' 'by' [0.1]
    DIV -> 'divided' 'by' [1.0]
""")

We can use the [nltk.CFG.productions()](https://www.nltk.org/api/nltk.html?highlight=production#nltk.grammar.CFG.productions) method to get a list of the PCFG's productions:

In [82]:
probabilistic_arithmetic_grammar.productions()

[S -> NUM [0.35],
 S -> NUM OP S [0.4],
 S -> S OP NUM [0.25],
 OP -> ADD [0.4],
 OP -> SUB [0.2],
 OP -> MULT [0.3],
 OP -> DIV [0.1],
 NUM -> 'one' [0.1],
 NUM -> 'two' [0.1],
 NUM -> 'three' [0.1],
 NUM -> 'four' [0.1],
 NUM -> 'five' [0.1],
 NUM -> 'six' [0.1],
 NUM -> 'seven' [0.1],
 NUM -> 'eight' [0.1],
 NUM -> 'nine' [0.1],
 NUM -> 'ten' [0.1],
 ADD -> 'plus' [0.8],
 ADD -> 'added' 'to' [0.2],
 SUB -> 'minus' [1.0],
 MULT -> 'times' [0.9],
 MULT -> 'multiplied' 'by' [0.1],
 DIV -> 'divided' 'by' [1.0]]

Each of the productions in the list is an instance of the [ProbabilisticProduction](https://www.nltk.org/api/nltk.html?highlight=production#nltk.grammar.ProbabilisticProduction) class. Each such instance is defined by three parameters: its left hand side (`lhs`), right-hand side (`rhs`), and rule probability (`prob`). These attributes can be accessed separately:

In [83]:
pprod_example = probabilistic_arithmetic_grammar.productions()[1]
print(f'For the production "{pprod_example}":\n' 
      f'left hand side of the rule is {pprod_example.lhs()}\n'
      f'right hand side of the rule is {pprod_example.rhs()}\n'
      f'probability of the rule is {pprod_example.prob()}')

For the production "S -> NUM OP S [0.4]":
left hand side of the rule is S
right hand side of the rule is (NUM, OP, S)
probability of the rule is 0.4


For non-probabilistic grammars, the class of productions is [Production](https://www.nltk.org/api/nltk.html?highlight=production#nltk.grammar.Production), which doesn't have a probability attribute and is only defined by its lhs and rhs attributes:

In [84]:
print(f'PCFG production: {probabilistic_arithmetic_grammar.productions()[1]} \n'
      f'      vs.\n'
      f'CFG production:  {arithmetic_grammar.productions()[1]}') 

PCFG production: S -> NUM OP S [0.4] 
      vs.
CFG production:  S -> NUM OP S


## Parse tree probabilities

To use a PCFG to select among parse trees, we need to be able to calculate the probability of a parse tree. The probability of a parse tree is simply the product of the probabilities of each constituent in the tree, the probability of the rule associated with the constituent.

You'll use the PCFG `probabilistic_arithmetic_grammar` to calculate the probability of each of the parse trees in `parses` (the list of trees which were parsed from the sentence "two plus three times four"). 

To do that, you'll need to get all the productions used in a parse tree (using the [productions](https://www.nltk.org/api/nltk.html?highlight=production#nltk.tree.Tree.productions) method), find their probabilities, and multiply them together.

First, we will create a dictionary from the PCFG, so that we can easily access the rule probabilities. Write a function which accepts a PCFG and returns a dictionary whose keys are the CFG (not PCFG) grammar rules and values are the associated probabilities. 
<!--
BEGIN QUESTION
name: pcfg_to_dict
-->

In [85]:
#TODO - returns a dictionary whose keys are `nltk.grammar.Production` objects
#       and whose values are the associated probabilities

def pcfg_to_dict(pcfg):
    d = {}
    for prod in pcfg.productions():
        (lhs, rhs, prob) = (prod.lhs(), prod.rhs(), prod.prob())
        d[nltk.grammar.Production(lhs,rhs)] = prob
    return d

In [86]:
grader.check("pcfg_to_dict")

All tests passed!

We can use the function you wrote to convert `probabilistic_arithmetic_grammar` to a dictionary and inspect it to make sure it's working.

In [87]:
pprint(pcfg_to_dict(probabilistic_arithmetic_grammar))

{ADD -> 'added' 'to': 0.2,
 ADD -> 'plus': 0.8,
 DIV -> 'divided' 'by': 1.0,
 MULT -> 'multiplied' 'by': 0.1,
 MULT -> 'times': 0.9,
 NUM -> 'eight': 0.1,
 NUM -> 'five': 0.1,
 NUM -> 'four': 0.1,
 NUM -> 'nine': 0.1,
 NUM -> 'one': 0.1,
 NUM -> 'seven': 0.1,
 NUM -> 'six': 0.1,
 NUM -> 'ten': 0.1,
 NUM -> 'three': 0.1,
 NUM -> 'two': 0.1,
 OP -> ADD: 0.4,
 OP -> DIV: 0.1,
 OP -> MULT: 0.3,
 OP -> SUB: 0.2,
 S -> NUM: 0.35,
 S -> NUM OP S: 0.4,
 S -> S OP NUM: 0.25,
 SUB -> 'minus': 1.0}


Now for the payoff: Write a function that takes a parse tree and a PCFG and returns the probability of the parse tree according to the PCFG. The `pcfg_to_dict` function you just wrote is likely to come in handy.

> Note that we are asking for the probability (not log probability), and we **don't work in log space** in this lab for simplicity, but for parse trees of longer sentences (which we'll see in the project) we might have to work in the log space to avoid underflows.

<!--
BEGIN QUESTION
name: parsed_trees_probs
-->

In [88]:
# TODO: returns the probability of the parse tree.
# `tree.productions() might be useful for getting the 
#  productions of a parse tree
def parse_probability(tree, pcfg):
    prod_to_prob = pcfg_to_dict(pcfg)
    tree_prob = 1
    for prod in tree.productions():
        tree_prob *= prod_to_prob[prod]
    return tree_prob


In [89]:
grader.check("parsed_trees_probs")

All tests passed!

We'll use it to calculate and print out the probability of each parse tree.

In [90]:
for i, tree in enumerate(parses):
    print(f'Probability of parsed tree {i+1} is '
          f'{parse_probability(tree, probabilistic_arithmetic_grammar):1.2e}')

Probability of parsed tree 1 is 3.02e-06
Probability of parsed tree 2 is 1.89e-06
Probability of parsed tree 3 is 4.84e-06
Probability of parsed tree 4 is 3.02e-06


<!-- BEGIN QUESTION -->

---
**Question:** Which of the trees is the most probable parse? Are there trees that got the same score (probability)? Do they have the same arithmetic result? Explain.

<!--
BEGIN QUESTION
name: open_response_ambiguity
manual: true
-->

*The most probable is the third. All the probabilities are different, even for trees with the same aritmetic results. It is expectable due to the fact that for S OP NUM and NUM OP S there are different probabilities, where the second is higher and all the other parts of the tree identically the same, what causes the third tree get a higher score.*

<!-- END QUESTION -->



---
## Estimating rule probabilities from a corpus

In the previous section, you received a CFG already augmented with rule probabilities. But where do rule probabilities come from? One way to generate rule probabilites is to learn them from a training corpus. 

In this section you will use a toy corpus of parsed sentences to generate maximum likelihood estimates of rule probabilities by counting the number of occurrences of a rule used in the corpus.

In [91]:
# The raw corpus, before splitting into separate phrases
corpus_raw = """
    (S (S (NUM two) (OP plus) (S (NUM six))) (OP times) (NUM one))
    (S (S (NUM eight) (OP minus) (S (NUM three))) (OP plus) (NUM seven))
    (S (NUM two) (OP plus) (S (S (NUM three)) (OP times) (NUM four))) 
    (S (S (NUM eight) (OP divided by) (S (NUM four))) (OP times) (NUM two))
    (S (S (NUM five) (OP divided by) (S (NUM two))) (OP plus) (NUM one))
    (S (NUM five) (OP minus) (S (NUM one) (OP times) (S (NUM four)))) 
    (S (S (NUM two) (OP times) (S (NUM three))) (OP plus) (NUM four))
    (S (S (S (NUM ten)) (OP minus) (NUM two)) (OP times) (NUM three))
"""

def corpus_from_string(raw):
  """Return a corpus as a list of sentences.
  
  The `raw` corpus is split at newlines, trimmed of whitespace, and blank 
  lines eliminated.
  """
  return list(filter(lambda x: x != '', 
                     map(lambda sent: sent.strip(),
                         raw.split('\n'))))

# The processed corpus we'll use
corpus = corpus_from_string(corpus_raw)

Recall that for the rule probabilities to define a valid probability distibution, the following needs to hold:

$$\sum_\beta p(\beta \given A) =1$$

Thus means that that after counting rule occurrences we will need to normalize them as

\begin{align}
p(\beta \given A) 
  &= \frac{\cnt{A \to \beta}}{\sum_{\beta'} \cnt{A \to \beta'}} \\
  &= \frac{\cnt{A \to \beta}}{\cnt{A}}
\end{align}

We will define three functions: 

1. `rule_counter` - accepts a list of sentences and returns a dictionary of rule counts (where the key is the production defined by the lhs and rhs and the value is the number of rule occurrences)
2. `lhs_counter` - accepts a list of sentences and returns a dictionary of lhs counts (where the key is the lhs nonterminal and the value is the count of that nonterminal's occurences as a lhs)
3. `rule_probs` - accepts a list of sentences and returns a dictionary of rule probabilities (where the key is the production and the value is the rule probability).

Implement these functions as specified above.

<!--
BEGIN QUESTION
name: probs_from_corpus
-->

In [92]:
#TODO 
from nltk.tree import Tree

def rule_counter(sentence_list):
    d = {}
    for sentence in sentence_list:
        prods = Tree.fromstring(sentence).productions()
        for prod in prods:
            d[prod] = d.get(prod, 0)+1
    return d

#TODO
def lhs_counter(sentence_list):
    d = {}
    for sentence in sentence_list:
        prods = Tree.fromstring(sentence).productions()
        for prod in prods:
            d[prod.lhs()] = d.get(prod.lhs(), 0)+1
    return d

#TODO
def rule_probs(sentence_list):
    d = {}
    left_dict = lhs_counter(sentence_list)
    rule_dict = rule_counter(sentence_list)
    for sentence in sentence_list:
        prods = Tree.fromstring(sentence).productions()
        for prod in prods:
            d[prod] = rule_dict[prod] / left_dict[prod.lhs()]
    return d

In [93]:
grader.check("probs_from_corpus")

All tests passed!

Now we can use the `rules_prob` function you wrote to get the rule probabilities from our corpus:

In [94]:
probs_from_corpus = rule_probs(corpus)
probs_from_corpus

{NUM -> 'eight': 0.08333333333333333,
 NUM -> 'five': 0.08333333333333333,
 NUM -> 'four': 0.16666666666666666,
 NUM -> 'one': 0.125,
 NUM -> 'seven': 0.041666666666666664,
 NUM -> 'six': 0.041666666666666664,
 NUM -> 'ten': 0.041666666666666664,
 NUM -> 'three': 0.16666666666666666,
 NUM -> 'two': 0.25,
 OP -> 'divided' 'by': 0.125,
 OP -> 'minus': 0.1875,
 OP -> 'plus': 0.3125,
 OP -> 'times': 0.375,
 S -> NUM: 0.3333333333333333,
 S -> NUM OP S: 0.3333333333333333,
 S -> S OP NUM: 0.3333333333333333}

Observe that the probabilities of the two rules `S -> NUM OP S` and `S -> S OP NUM` are equivalent. **Modify** the `corpus` defined above such that the rule "S -> NUM OP S" will get a higher probability than "S -> S OP NUM". Call your new version `corpus_new`. Remember that `corpus_new` should be a list of strings, not a single string (like `corpus_raw` above).

<!--
BEGIN QUESTION
name: change_corpus
-->

In [98]:
corpus_new = """
    (S (NUM one) (OP times) (S (NUM two) (OP plus) (S (NUM six))))
    (S (S (NUM eight) (OP minus) (S (NUM three))) (OP plus) (NUM seven))
    (S (NUM two) (OP plus) (S (S (NUM three)) (OP times) (NUM four))) 
    (S (S (NUM eight) (OP divided by) (S (NUM four))) (OP times) (NUM two))
    (S (S (NUM five) (OP divided by) (S (NUM two))) (OP plus) (NUM one))
    (S (NUM five) (OP minus) (S (NUM one) (OP times) (S (NUM four)))) 
    (S (S (NUM two) (OP times) (S (NUM three))) (OP plus) (NUM four))
    (S (S (S (NUM ten)) (OP minus) (NUM two)) (OP times) (NUM three))
"""


corpus_new = corpus_from_string(corpus_new)

check = """
for sentence in new_corpus:
    t = Tree.fromstring(sentence)
    t.pretty_print()

probs_from_corpus = rule_probs(corpus_new)
probs_from_corpus
"""

In [96]:
grader.check("change_corpus")

All tests passed!

<!-- BEGIN QUESTION -->

---
**Question:** The example that we provided of an ambiguity introduced by multiple productions and disambiguated by their probabilities – the multiple rules for arithmetic expressions – is admittedly quite artificial. Can you think of other (more natural) examples, in natural language or elsewhere, where this phenomenon might occur?

<!--
BEGIN QUESTION
name: open_response_other_examples
manual: true
-->

Some examples of CFG where this phenomenon miht occur:
* The man saw the woman with an telescope.
   
   The ambiguity : did he saw her when he was looking through a telescope or that she was carring a telescope with her?
* Large hole appears in Main Street. City officials are looking into it.
    
    The ambiguity : are the city officials looking into the hole or looking for the hole thats appears in the city?

Additional exmaples which were presented in the *reading material*:

In Groucho Marx’s well-known line as Captain Spaulding in "Animal Crackers":

* "I shot an elephant in my pajamas" can be interpretated in ambiguous way:

1. Less semantically probable, but structurally possible, the elephant is in Captain Spaulding's pijamas.
2. The correct interpretation: Captain Spaulding was in his pijamas when he shot the elephant.

* "We saw the Eiffel Tower flying to Paris", where "flying" can be related to "We" or to the Tower.

*Additional kind of examples:*
For the connectors like "and" we can observe the following ambuigiuty:
*"old men and women": can be interpretated as only the men are old, as much as both old man and women.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

---

## Lab debrief – for consensus submission only

**Question:** We're interested in any thoughts your group has about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on include the following: 

* Was the lab too long or too short?
* Were the readings appropriate for the lab? 
* Was it clear (at least after you completed the lab) what the points of the exercises were? 
* Are there additions or changes you think would make the lab better?

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# End of Lab 3-3

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [97]:
grader.check_all()

change_corpus:

    All tests passed!
    

parsed_equation_result:

    All tests passed!
    

parsed_trees_probs:

    All tests passed!
    

pcfg_to_dict:

    All tests passed!
    

probs_from_corpus:

    All tests passed!